<a href="https://colab.research.google.com/github/blackbaba980/GameOfThronesPredictions/blob/main/code/GameOfThronesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
!pip3 install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.15-cp37-cp37m-linux_x86_64.whl size=516862 sha256=a27a611c6c01b56b0c597dbec85e252da3334ce324ec5715b7329eb558d6fb7c
  Stored in directory: /root/.cache/pip/wheels/15/68/4d/1414be5c2c622bad35364e13213180797717b6d4b8923936dc
Successfully built torch-sparse


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


import math
import time

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

random_seed = 7
#set the random seeds 
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

epochs = 200
lr = 0.01
weight_decay = 5e-4
hidden_units = 16
dropout = 0.5

# Dataset Preprocessing


In [67]:
#per season adjaceny matrix, features of every node


def load_data(season_no):

  #read the edge and node files
  dir = "./data/"
  edge_file = dir + "got-s" + str(season_no) + "-edges.csv"
  node_file = dir + "got-s" + str(season_no) + "-m-nodes.csv"

  edges = pd.read_csv(edge_file)
  nodes = pd.read_csv(node_file)
  
  
  #only get those edges where the nodes are present
  node_ids = nodes['CHARACTERS ID']
  node_ids_reverse = node_ids.to_dict()
  node_ids = {v: k for k, v in node_ids_reverse.items()}
  
  #remove unnecessary columns
  features = nodes.drop(['CHARACTERS ID'], axis=1)
  features = (features-features.mean())/features.std()
  features = torch.from_numpy(features.to_numpy().astype(np.float32))
  edges = edges.drop(['Season'], axis=1)
  
  #Adjacency matrix
  gr = nx.Graph()
  adj = np.zeros( (len(node_ids),len(node_ids)) )

  for i in range(len(edges)):
    if edges.iloc[i]['Source'] in node_ids and edges.iloc[i]['Target'] in node_ids:
      src_index = node_ids[edges.iloc[i]['Source']]
      dst_index = node_ids[edges.iloc[i]['Target']]
      gr.add_edge(src_index, dst_index, weight=edges.iloc[i]['Weight'])
      adj[src_index][dst_index] = edges.iloc[i]['Weight']


  adj = torch.from_numpy(adj.astype(np.float32))


  return features, adj, gr, node_ids

  #return torch.from_numpy(features.to_numpy()), torch.from_numpy(adj), gr, node_ids
  #return nodes, edges



features, adj, graph, node_ids = load_data(1)


# Per Season Graph

# Generative Model

In [91]:
class GraphConvolution(Module):
  def __init__(self, in_features, out_features, bias=True):
    super(GraphConvolution, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = Parameter(torch.FloatTensor(in_features, out_features))
    if bias:
        self.bias = Parameter(torch.FloatTensor(out_features))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

  def reset_parameters(self):
    stdv = 1. / math.sqrt(self.weight.size(1))
    self.weight.data.uniform_(-stdv, stdv)
    if self.bias is not None:
        self.bias.data.uniform_(-stdv, stdv)

  def forward(self, input, adj):
    support = torch.mm(input, self.weight)
    output = torch.mm(adj, support)
    if self.bias is not None:
        return output + self.bias
    else:
        return output

  def __repr__(self):
    return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'

In [100]:
class Encoder(nn.Module):
  def __init__(self, nfeat, nhid, nclass, dropout):
    super(Encoder, self).__init__()

    self.conv = GraphConvolution(nfeat, nhid)
    self.mu = GraphConvolution(nhid, nclass)
    self.sig = GraphConvolution(nhid, nclass)    

    
    self.dropout = dropout
    self.embeddings = []

  def forward(self, x, adj, save_embeddings = False):
    x = F.relu(self.conv(x, adj))
    mu = self.mu(x, adj)
    logvar = self.sig(x, adj)
    return mu, logvar
    

In [108]:
from torch_geometric.nn.models import InnerProductDecoder, VGAE

In [188]:
class Decoder(nn.Module):
  def __init__(self, latent_dim, output_dim):
    super(Decoder, self).__init__()
    self.decoder = InnerProductDecoder()
    


  def forward_all(self, x, sigmoid = 'True'):
    #first find whether there is edge or not => THis would be the reconstruction loss
    #then find the weights
    y = self.decoder.forward_all(x, sigmoid = sigmoid)
    return y

  def forward(self, x, edge_index , sigmoid = 'True'):
    y = self.decoder(x, edge_index, sigmoid = sigmoid)
    return y





In [209]:
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops

class DeepVGAE(VGAE):
  def __init__(self):
    super(DeepVGAE, self).__init__(encoder=Encoder(10, 20, 1, dropout), decoder= Decoder(42,1))


  def forward(self, x, adj):
    z = self.encode(x, adj)
    adj_pred = self.decoder.forward_all(z)
    return adj_pred

  def loss(self, x, adj, pos_edge_index, all_edge_index):
    z = self.encode(x, adj)

    kl_loss = 1 / x.size(0) * self.kl_loss()

    return self.recon_loss(z, pos_edge_index) + kl_loss



In [210]:
enc = Encoder(10, 20, 1, dropout)

In [211]:
model = DeepVGAE()

In [212]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [213]:
out = model(features, adj)


In [214]:
l = model.loss(features, adj, pyg_graph.edge_index, pyg_graph.edge_index)
print(l)
l.backward()

tensor(1983913.3750, grad_fn=<AddBackward0>)


In [215]:
optimizer.step()

In [216]:
from torch_geometric.utils.convert import from_networkx
pyg_graph = from_networkx(graph)

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  6,
          6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
          8,  8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
         11, 11, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
         13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18,
         18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20,
         20, 20, 20, 20, 20,